In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)

In [3]:
current_date = dt.date.today()
day = current_date.day
month = current_date.month
year = current_date.year

way = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Илларионов\Выгрузка Еком\\'

In [4]:
data_IM = pd.read_excel(rf'{way}\{month}.{year}_продажи еком.xlsx')

In [5]:
# смотрим продажи с последнего мониторинга
data = dt.datetime.isoweekday(dt.date.today())
if data == 1:
    last_monitoring = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
else:
    last_monitoring = (dt.date.today() - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    
data_IM = data_IM[data_IM['created'] >= f'{last_monitoring}']

In [6]:
data_IM.sample(5)

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana
11607,af60036f-90e2-4b36-aa09-001024ceaa30,2024-05-27 21:48:50.810,Кожаный шнурок с серебряным замком,done,79604600001.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1173160,6-01-015,3.27,1.07,NaN,925,СИ ЦБ ШНУР,ЦЕПЬ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ ЦЕПЬ,NaN,38-72-00B0-28668,2405272148261,784.00,WEB,2024-05-28 15:28:59.681,NaT,8358007,365.00,2078609222222,6-01-015,274,0.57,156.12,627.88
4310,40c780c4-3529-4447-a196-34ed0d214aed,2024-05-27 20:39:57.992,Серебряная подвеска буква А с фианитами и роди...,done,79954119846.00,Katya16-94@mail.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1215549,6-02-008,0.56,0.52,NaN,925,СИ БУКВЫ,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ ПОДВЕС ИФ,NaN,306176-301-0019,2405272039627,406.00,WEB,2024-05-28 14:16:51.232,NaT,1870369,135.30,2078609658473,6-02-008,240,0.34,80.93,325.07
10284,9c1d6649-b08d-40f0-82f7-b73c9d4d61a3,2024-05-28 07:51:48.077,Золотое гладкое обручальное кольцо 5 мм,done,79961830867.00,al.mazurenko15@mail.ru,done,PICKUP_NOW,YANDEX_KASSA,3d84ba8b-6021-4bfa-8bf0-8f3e3f6880ba,SBER,[YANDEX_KASSA] SBER - URL,ТОВ0963683,4-01-064,4.97,4.97,NaN,585,ИФ ОБРУЧ ГЛАДКАЯ КОМФОРТ,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,КОМ 05-00,2405280751300,28370.00,MOBILE,2024-05-28 07:53:57.754,NaT,2576554,13264.16,2078533692786,4-01-064,0,0.00,0.00,28370.00
11748,b1b5203c-bf80-4079-b55a-b9bb2d38308b,2024-05-27 10:48:13.533,Золотая полновесная цепочка Круглый якорь,done,79127944102.00,NaN,done,PICKUP_NOW,CASHIER,NaN,CASHIER,[CASHIER] CASHIER - CASHIER,ТОВ01258924,4-07-023,1.22,1.22,NaN,375,ЯКОРЬ,ЦЕПЬ,КРАСНЫЙ,ЦБ ЦЕПИ,NaN,310-90-0025-30555,2405271048469,4860.00,MOBILE,2024-05-27 13:53:33.060,NaT,7977021,2560.18,2078609624979,4-07-023,0,0.00,0.00,4860.00
10262,9bc338ba-594f-4d00-97d5-dcd3af183602,2024-05-28 09:35:27.910,Золотая пустотелая цепочка Бисмарк с алмазной ...,done,79235578552.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ01260756,3-03-033,4.20,4.20,NaN,585,БИСМАРК ПУСТ,ЦЕПЬ,КРАСНЫЙ,ЦБ ЦЕПИ,ADRIATICA,331-01-0050-37654,2405280935596,39195.00,WEB,2024-05-28 09:38:28.562,NaT,2246821,13913.50,2078609169166,3-03-033,7839,1.00,7839.00,31356.00


In [7]:
data_IM = data_IM.drop_duplicates()
data_IM = data_IM.reset_index(drop=True)

data_IM = data_IM[(data_IM['state_order'] != 'returned') & (data_IM['state_order'] != 'canceled')]
data_IM = data_IM[~((data_IM['delivery_type'] == 'PICKUP_NOW') & (data_IM['payment_type'] == 'CASHIER') & (data_IM['type_ibox'] == 'CASHIER'))|
                ((data_IM['delivery_type'] == 'PICKUP') & (data_IM['payment_type'] == 'CASHIER') & (data_IM['type_ibox'] == 'CASHIER'))]

In [8]:
data_IM.sample(5)

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana
523,f8ff421c-37c0-44d6-8ff1-56b65a1620cf,2024-05-27 15:28:11.153,Серебряный браслет Бисмарк с чернением,done,79019884471.00,voroncova1219@icloud.com,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1228495,2-09-061,9.82,9.82,NaN,925,СИ ЦБ БИСМАРК РУЧНОЙ,БРАСЛЕТ,ЧЁРНЫЙ,СИ БРАСЛЕТ БК,NaN,36999022Ч,2405271528534,4220.00,MOBILE,2024-05-27 16:03:50.652,NaT,1578237,1395.32,2078609202027,2-09-061,844,1.00,844.00,3376.00
235,76da16bc-6acc-4768-b3e3-7adb37ce2720,2024-05-27 15:14:25.024,Золотая цепочка Нонна,done,79273719674.00,s.oksana.85@mail.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1269131,5-02-070,3.11,3.11,NaN,585,НОННА,ЦЕПЬ,КРАСНЫЙ,ЦБ ЦЕПИ,NaN,ЦН235А2-А51#ТД,2405271514374,22130.00,MOBILE,2024-05-27 15:17:52.109,NaT,1081499,11389.94,2078610164560,5-02-070,6270,0.71,4426.00,17704.00
127,3cb24d47-8147-4a38-947c-2945052ec95f,2024-05-28 10:19:05.365,Золотое кольцо с фианитами,done,79826470877.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ1187058,4-01-062,0.63,0.59,NaN,585,ИФ ЛЕПЕСТКИ,КОЛЬЦО,КРАСНЫЙ,ИФ КОЛЬЦА,NaN,Z1-1996-11-01,2405281019726,4875.00,MOBILE,2024-05-28 10:32:44.792,NaT,1297417,1511.20,2078556037434,4-01-062,975,1.00,975.00,3900.00
437,cf4841db-36ff-43c6-b62f-ab08ac3d55a4,2024-05-27 08:23:34.218,Серебряная цепочка Панцирь с алмазной гранью,done,79088300883.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ1203246,2-09-149,1.82,1.82,NaN,925,СИ ЦБ ПАНЦИРЬ,ЦЕПЬ,БЕЛЫЙ,СИ ЦЕПЬ,NaN,40-51035-22,2405270823950,392.00,MOBILE,2024-05-28 18:22:56.190,NaT,1629757,225.54,2078610307381,2-09-149,688,0.11,78.31,313.69
528,fb1e9139-cbe2-4171-b2b6-1858dc951b9d,2024-05-28 06:00:09.294,Золотые серьги,done,79036986796.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1181163,2-09-124,2.02,2.02,NaN,585,ИФ ШТАМП,СЕРЬГИ,КРАСНЫЙ,БК СЕРЬГИ,NaN,18-02-0000-28903#ТД,2405280600412,19838.00,MOBILE,2024-05-28 16:09:26.656,NaT,1037416,6710.66,2078609841051,2-09-124,3967,1.00,3967.00,15871.00


In [9]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'], format = '%Y-%m-%d')

In [10]:
def str_float(row):
    row = str(row)
    row = row.replace(',','.')
    return row

data_IM['price'] = data_IM['price'].apply(str_float)
data_IM['price'] = data_IM['price'].astype(float)

data_IM['avg_net_weight'] = data_IM['avg_net_weight'].apply(str_float)
data_IM['avg_net_weight'] = data_IM['avg_net_weight'].astype(float)

In [11]:
data_IM = data_IM[data_IM['price'] > 1].reset_index(drop=True)
data_IM = data_IM[data_IM['avg_net_weight'] > 0].reset_index(drop=True)

In [12]:
data_IM['Товарное направление'] = np.where(data_IM['tg'].isin(['БК ПОДВЕС КУЛЬТ', 'БК СЕРЬГИ', 'БК КОЛЬЦА','БК КОЛЬЦА ОБРУЧ','БК ПОДВЕС ДЕКОР',  'БК ПЕЧАТКИ','БК АКСЕССУАРЫ','ОПТ БК']), 'БК',
                                np.where(data_IM['tg'].isin(['ПДК СЕРЬГИ', 'ПДК КОЛЬЦА','ПДК ПОДВЕСКИ']), 'ПДК',
                                np.where(data_IM['tg'].isin(['ДК СЕРЬГИ',  'ДК КОЛЬЦА',    'ДК ПОДВЕСКИ',    'ДК КОЛЬЦА ОБРУЧ']), 'ДК',
                                np.where(data_IM['tg'].isin(['ИФ СЕРЬГИ',  'ИФ КОЛЬЦА', 'ИФ ПОДВЕС ДЕКОР', 'ИФ ПЕЧАТКИ', 'ИФ ПОДВЕС КУЛЬТ','ИФ КОЛЬЦА ОБРУЧ','ИФ АКСЕССУАРЫ']), 'ИФ',
                                np.where(data_IM['tg'].isin(['ЦБ ЦЕПИ', 'ЦБ БРАСЛЕТЫ']), 'ЦБ',
                                np.where(data_IM['tg'].isin(['СИ ПОДВЕС БК','СИ КОЛЬЦО ИФ','СИ БРАСЛЕТ БК','СИ КОЛЬЦО БК','СИ ЦЕПЬ','СИ СЕРЬГИ БК','СИ СЕРЬГИ ИФ','СИ ПОДВЕС ИФ','СИ СЕРЬГИ РАЗН','СИ СЕРЬГИ ЦВ/К',\
                                                            'СИ КОЛЬЦО ЦВ/К','','СИ ШАРМЫ','СИ ОБРУЧАЛЬНОЕ','СИ КОЛЬЦО ПЕЧАТ','СИ ПОДВЕСЦВ/К','СИ БРАСЛЕТ СК','СИ ЖЕМЧУГ',\
                                                            'СИ АКСЕССУАР','СИ ДФРС ПРОЕКТ','СИ КЕРАМИКА','СИ ПОСУДА','СИ ЯНТАРЬ','СИ ЭМАЛЬ','СИ ВЫРИЦА']), 'СИ',
                                np.where(data_IM['tg'].isin(['ЧАСЫ']), 'ЧАСЫ','другое')))))))

data_IM['Товарное направление'].value_counts()

Товарное направление
СИ        115
БК        107
ИФ         85
ЦБ         64
ПДК        30
ДК         28
другое     18
Name: count, dtype: int64

In [13]:
proverka = data_IM[data_IM['Товарное направление']=='другое']
proverka['tg'].unique()

array(['СИ СЕРЬГИ\xa0 ЦВ/К', 'ОПТ ИФ', 'ДК ПЕЧАТКИ',
       'СИ БРАСЛЕТ БК Декор', 'СИ ПОДВЕС\xa0ЦВ/К', 'СИ БРАСЛЕТ ЦВ/К',
       'ВЫРИЦА СЕРЕБРО', 'ВЫРИЦА ЗОЛОТО'], dtype=object)

In [14]:
data_IM = data_IM[~(data_IM['name'].str.contains('каучук')
        |data_IM['name'].str.contains('шнурок')
        |data_IM['name'].str.contains('текстиль')
        |data_IM['name'].str.contains('хлопок')
        |data_IM['name'].str.contains('шелк')
        |data_IM['name'].str.contains('на нитке')
        |data_IM['name'].str.contains('кожа')
        |data_IM['name'].str.contains('жемчуг'))]

In [15]:
data_IM.info()

<class 'pandas.core.frame.DataFrame'>
Index: 436 entries, 0 to 446
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         436 non-null    object        
 1   created                    436 non-null    datetime64[ns]
 2   name                       436 non-null    object        
 3   state_order                436 non-null    object        
 4   recipient_phone            435 non-null    float64       
 5   recipient_email            151 non-null    object        
 6   state_item                 436 non-null    object        
 7   delivery_type              436 non-null    object        
 8   payment_type               436 non-null    object        
 9   employee_id                46 non-null     object        
 10  type_ibox                  436 non-null    object        
 11  type_name                  436 non-null    object        
 12  tov          

In [16]:
data_IM = data_IM[(data_IM['Товарное направление'].isin(['БК', 'ЦБ']))&(data_IM['proba'] == '585')]

In [17]:
data_IM['Товарное направление'].value_counts()

Товарное направление
БК    88
ЦБ    55
Name: count, dtype: int64

In [18]:
data_IM['proba'].unique()

array(['585'], dtype=object)

In [19]:
data_IM['date_status_order_done'].value_counts()

date_status_order_done
2024-05-28 15:55:28.501    1
2024-05-28 05:36:50.354    1
2024-05-28 06:31:39.415    1
2024-05-27 13:03:37.198    1
2024-05-27 08:08:18.361    1
                          ..
2024-05-27 09:28:11.109    1
2024-05-27 12:20:50.440    1
2024-05-27 12:44:36.114    1
2024-05-27 12:44:36.190    1
2024-05-28 06:19:43.021    1
Name: count, Length: 143, dtype: int64

In [20]:
previous_date = data_IM['date_status_order_done'].min()

In [21]:
filtr_date = data_IM['date_status_order_done'].max()
print(filtr_date)

2024-05-28 17:41:23.687000


In [22]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'], format='%Y-%m-%d')

In [23]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [24]:
data_today = data_IM.query("date_status_order_done >= @previous_date")
data_today.head()

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana,Товарное направление
14,082dbfd6-5a8f-4d5b-9612-b99e9e7f36ab,2024-05-28 15:46:22.423,Золотая подвеска Икона Святой Матроны Московской,done,79064898805.00,notabak@ya.ru,done,PICKUP_NOW,YANDEX_KASSA,556dace1-0d82-473b-84f9-e53534bcba89,YANDEX_PAY,Яндекс пэй для мп,ТОВ1095794,6-05-037,0.80,0.80,NaN,585,ИФ ИКОНА,ПОДВЕС КУЛЬТОВЫЙ,КРАСНЫЙ,БК ПОДВЕС КУЛЬТ,NaN,35-0469-11-64,2405281546480,5670.00,MOBILE,2024-05-28,NaT,7182769,1884.76,2078602760663,6-05-037,0,0.00,0.00,5670.00,БК
19,09961878-3d7e-413f-b77a-7c33df93bb24,2024-05-28 15:33:30.475,Золотое кольцо,done,79202997750.00,alopex@inbox.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ1142395,2-09-065,3.29,3.29,NaN,585,ИФ ПРОЧЕЕ,КОЛЬЦО,КРАСНЫЙ,БК КОЛЬЦА,NaN,20-00-0000-27912,2405281533709,19190.00,MOBILE,2024-05-28,NaT,2239924,7410.77,2078496042277,2-09-065,0,0.00,0.00,19190.00,БК
21,0bb6b795-8414-4ecf-bc40-5d6d8334af5c,2024-05-28 09:33:04.236,Золотой пустотелый браслет Нонна,done,79236343905.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,YANDEX_PAY,Яндекс пэй для сайта,ТОВ0964014,3-02-003,1.44,1.44,NaN,585,НОННА ПУСТ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,NaN,"НБ12-200ПГ D0,5",2405280933453,14313.00,WEB,2024-05-28,NaT,3928876,6237.02,2078607190807,3-02-003,0,0.00,0.00,14313.00,ЦБ
35,1693e0af-346d-4be6-abda-5d6483cc7923,2024-05-27 14:24:28.777,Золотой пустотелый браслет Двойной ромб с алма...,done,79269887601.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ01262098,2-09-139,0.74,0.74,NaN,585,РОМБ ДВОЙНОЙ ПУСТ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,NaN,41-01-0035-30176,2405271424723,4900.00,MOBILE,2024-05-28,NaT,1674395,2482.56,2078609010442,2-09-139,0,0.00,0.00,4900.00,ЦБ
52,238dc957-f3e0-471b-a5e7-6390870e88e0,2024-05-28 07:10:33.839,Золотая печатка,done,79881118838.00,anastasiahvostova88@gmail.com,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1190953,6-05-019,4.97,4.97,NaN,585,ИФ ПЕЧАТКА,КОЛЬЦО ПЕЧАТКА,КРАСНЫЙ,БК ПЕЧАТКИ,NaN,21-02-0000-29330,2405280710667,39455.00,MOBILE,2024-05-28,NaT,1131533,16815.35,2078606001762,6-05-019,7891,1.00,7891.00,31564.00,БК


In [25]:
data_today['date_status_order_done'].unique()

<DatetimeArray>
['2024-05-28 00:00:00']
Length: 1, dtype: datetime64[ns]

In [26]:
data_today.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, 14 to 446
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         84 non-null     object        
 1   created                    84 non-null     datetime64[ns]
 2   name                       84 non-null     object        
 3   state_order                84 non-null     object        
 4   recipient_phone            84 non-null     float64       
 5   recipient_email            24 non-null     object        
 6   state_item                 84 non-null     object        
 7   delivery_type              84 non-null     object        
 8   payment_type               84 non-null     object        
 9   employee_id                12 non-null     object        
 10  type_ibox                  84 non-null     object        
 11  type_name                  84 non-null     object        
 12  tov          

In [27]:
data_today['цена за гр'] = data_today['price_wo_manzana'] / data_today['avg_weight']
data_today['цена за гр'].describe()

count      84.00
mean     6799.15
std      1442.21
min      4380.95
25%      6110.00
50%      6681.85
75%      7592.14
max     15024.59
Name: цена за гр, dtype: float64

In [28]:
data_today['avg_weight'].describe()

count   84.00
mean     2.57
std      1.81
min      0.19
25%      1.17
50%      2.25
75%      3.32
max      8.84
Name: avg_weight, dtype: float64

In [29]:
data_today = data_today[~data_today['tg'].str.contains('ОПТ БК|БК ПЕЧАТКИ|ОПТ КОНГО')]

In [30]:
data_today = data_today[~data_today['client_type'].str.contains('OZON')]

In [31]:
print('Средняя цена продажи еком: ', (data_today['price_wo_manzana'].sum()/data_today['avg_weight'].sum()).round(0))

Средняя цена продажи еком:  6493.0


In [32]:
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')

In [33]:
data_today.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\ecom-sales\online_sales_{date_now}.xlsx', index=False)

In [33]:
tn = data_today.groupby('tg', as_index=False)[['price_wo_manzana', 'avg_weight', 'cost_price']].agg('sum')
tn['price_gramm'] = (tn['price_wo_manzana'] / tn['avg_weight']).round(0)

In [34]:
tn['price_gramm'] = tn['price_gramm'].astype('int')

In [35]:
tn['Наценка продажи онлайн'] =  tn['price_wo_manzana'] / tn['cost_price'] - 1

In [36]:
tn

,tg,price_wo_manzana,avg_weight,cost_price,price_gramm,Наценка продажи онлайн
0,БК КОЛЬЦА,39938.00,6.68,14191.08,5979,1.81
1,БК КОЛЬЦА ОБРУЧ,439458.00,72.04,186500.51,6100,1.36
2,БК ПОДВЕС ДЕКОР,20223.00,2.73,9297.43,7408,1.18
3,БК ПОДВЕС КУЛЬТ,61989.37,10.22,27448.44,6065,1.26
4,БК СЕРЬГИ,117423.00,17.31,55505.80,6784,1.12
5,ЦБ БРАСЛЕТЫ,251610.00,39.19,128833.26,6420,0.95
6,ЦБ ЦЕПИ,439303.00,62.82,200759.01,6993,1.19


In [37]:
print('Средняя цена продажи еком: ', (tn['price_wo_manzana'].sum() / tn['avg_weight'].sum()).round(0))

Средняя цена продажи еком:  6493.0


In [39]:
tn.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\ecom-sales\ecom_sales_tg\\ecom_sales_{date_now}.xlsx', index=False)

In [38]:
tn = tn.drop(columns=['price_wo_manzana', 'avg_weight', 'cost_price', 'Наценка продажи онлайн'])

In [39]:
tn.columns = ['tg', '585*Золотой онлайн продажи']

In [40]:
day

29

In [43]:
pd.read_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx').merge(tn, how = 'left', on = 'tg').to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx', index=False)